In [1]:
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd.profiler as profiler

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


LOG_PATH = "./data/profile_info/"
GRAPH_PATH = "./data/graphs/train.jpg"
DATA_DIR = './data/MNIST'

In [2]:
import sys 
sys.path.append("..")
from train_module.training_functions import train, show_plot

In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [4]:
def load_data(data_dir, batch_size):
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [5]:
in_channels = 3
out_channels = 10
device = "cuda:0"

train_loss = nn.CrossEntropyLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 50

In [6]:
trainset, testset = load_data(data_dir=DATA_DIR, batch_size=batch_size)

In [7]:
class Network(nn.Module):
  def __init__(self, channels_list):
    super().__init__()
    channels_list.insert(0, 1)
    self.convs = nn.ModuleList([
        nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )        
        for in_ch, out_ch in zip(channels_list[:-1], channels_list[1:])
    ])
    self.out = nn.Linear(in_features=channels_list[-1], out_features=10)

  def forward(self, tensor):
    for layer in self.convs:
        tensor = layer(tensor)
        tensor = F.max_pool2d(tensor, kernel_size=2) 

    # output
    tensor = nn.AdaptiveAvgPool2d((1,1))(tensor) 
    tensor = torch.flatten(tensor, 1)
    tensor = self.out(tensor)
    tensor = torch.sigmoid(tensor)

    return tensor

In [8]:
net = Network([8, 16, 32, 64])
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [9]:
net, stats = train(
                net, optimizer, train_loss, epoch_num, trainset, testset, device, GRAPH_PATH
)

100%|██████████| 50/50 [15:32<00:00, 18.64s/it]
